## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-28-09-00-00 +0000,cbc,How are the NDP leadership hopefuls different?...,https://www.cbc.ca/news/politics/ndp-leadershi...
1,2026-01-28-08-56-00 +0000,wsj,"Oil Extends Gains on Iran Risks, U.S. Winter S...",https://www.wsj.com/finance/commodities-future...
2,2026-01-28-08-50-00 +0000,wsj,Euro Could Rise Further Versus Weak Dollar,https://www.wsj.com/finance/currencies/yen-wea...
3,2026-01-28-08-43-43 +0000,bbc,South Korea's ex-first lady jailed for 20 mont...,https://www.bbc.com/news/articles/cevnppplkjjo...
4,2026-01-28-08-40-00 +0000,wsj,"Gold Extends Rally, Surging Past $5,200 as Dol...",https://www.wsj.com/finance/commodities-future...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2483/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
49,trump,70
388,ice,23
132,minnesota,19
56,year,17
108,minneapolis,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
43,2026-01-28-04-36-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...,153
328,2026-01-27-14-15-00 +0000,wsj,President Trump’s pivot in Minneapolis came af...,https://www.wsj.com/politics/policy/the-48-hou...,147
352,2026-01-27-12-02-00 +0000,wsj,Less than 48 hours after U.S. agents fatally s...,https://www.wsj.com/politics/policy/homan-in-b...,147
34,2026-01-28-05-38-48 +0000,bbc,Trump says government will 'de-escalate' in Mi...,https://www.bbc.com/news/articles/c0q425vg4qzo...,138
85,2026-01-28-02-00-00 +0000,wsj,Trump is looking for someone to run the Fed wh...,https://www.wsj.com/economy/central-banking/tr...,137


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
43,153,2026-01-28-04-36-00 +0000,wsj,President Trump defended Homeland Security Sec...,https://www.wsj.com/politics/policy/rump-krist...
209,79,2026-01-27-20-11-28 +0000,nyt,Photo Shows ICE Agents Deploying at Minneapoli...,https://www.nytimes.com/2026/01/27/us/minneapo...
85,67,2026-01-28-02-00-00 +0000,wsj,Trump is looking for someone to run the Fed wh...,https://www.wsj.com/economy/central-banking/tr...
298,67,2026-01-27-15-59-40 +0000,nypost,Meta and YouTube face landmark trial over yout...,https://nypost.com/2026/01/27/business/meta-ti...
328,52,2026-01-27-14-15-00 +0000,wsj,President Trump’s pivot in Minneapolis came af...,https://www.wsj.com/politics/policy/the-48-hou...
339,51,2026-01-27-13-22-00 +0000,wsj,EU and India Reach Free-Trade Deal as World Re...,https://www.wsj.com/economy/trade/eu-and-india...
27,49,2026-01-28-06-26-29 +0000,nypost,South Korea’s former first lady Kim Keon Hee s...,https://nypost.com/2026/01/28/world-news/south...
100,48,2026-01-28-01-24-32 +0000,nypost,Rep. Ilhan Omar sprayed with unknown substance...,https://nypost.com/2026/01/27/us-news/rep-ilha...
247,46,2026-01-27-18-23-00 +0000,wsj,Amazon to Shut Down All Amazon Go and Amazon F...,https://www.wsj.com/business/retail/amazon-to-...
228,43,2026-01-27-19-41-00 +0000,wsj,Bari Weiss Lays Out New Vision for CBS News,https://www.wsj.com/business/media/bari-weiss-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
